In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

# Define the directory containing the images
image_dir = 'extracted_images/images'

# Create a DataFrame with image paths and labels
data = []
for label in os.listdir(image_dir):
    label_dir = os.path.join(image_dir, label)
    if os.path.isdir(label_dir):
        for file in os.listdir(label_dir):
            if file.endswith(('png', 'jpg', 'jpeg')):
                data.append([os.path.join(label_dir, file), label])

df = pd.DataFrame(data, columns=['image_path', 'label'])
print(df.head())


ImportError: dlopen(/Users/syedather/opt/anaconda3/envs/orel/lib/python3.10/site-packages/tensorflow/python/_pywrap_tfe.so, 0x0002): Library not loaded: @rpath/_pywrap_tensorflow_internal.so
  Referenced from: <25FF818B-3DDE-3FCD-BF69-B005228278D6> /Users/syedather/opt/anaconda3/envs/orel/lib/python3.10/site-packages/tensorflow/python/_pywrap_tfe.so
  Reason: tried: '/Users/syedather/opt/anaconda3/envs/orel/lib/python3.10/site-packages/tensorflow/python/../../_solib_darwin_x86_64/_U_S_Stensorflow_Spython_C_Upywrap_Utfe.so___Utensorflow/_pywrap_tensorflow_internal.so' (no such file), '/Users/syedather/opt/anaconda3/envs/orel/lib/python3.10/site-packages/tensorflow/python/../../_solib_darwin_x86_64/_U_S_Stensorflow_Spython_C_Upywrap_Utensorflow_Uinternal_Umacos___Utensorflow_Spython/_pywrap_tensorflow_internal.so' (no such file), '/Users/syedather/opt/anaconda3/envs/orel/lib/python3.10/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so' (no such file), '/Users/syedather/opt/a

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Create train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Display the distribution of labels in train and test sets
print(train_df['label'].value_counts())
print(test_df['label'].value_counts())

# Data augmentation and normalization
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df, x_col='image_path', y_col='label',
                                                    class_mode='raw', target_size=(224, 224), batch_size=32)

test_generator = test_datagen.flow_from_dataframe(dataframe=test_df, x_col='image_path', y_col='label',
                                                  class_mode='raw', target_size=(224, 224), batch_size=32)


In [ ]:
model_1 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_1.summary()

history_1 = model_1.fit(train_generator, epochs=10, validation_data=test_generator)
